In [1]:
import os
import pandas as pd
import seaborn as sns

import sys  # noqa
sys.path.append('../..')  # noqa

from utils.data_paths import make_data_list
from dataclasses import dataclass, field, asdict
from datetime import datetime, timedelta
from typing import Tuple
import re 

In [2]:
@dataclass
class OutputLogs:
    """ Holds the paths to the output logs for each pipeline. """
    bio3: str = ""
    bio4: str = ""
    jams: list[str] = field(default_factory=list)
    wgsa2: str = ""
    woltka: list[str] = field(default_factory=list)
    sunbeam: str = ""


@dataclass
class TimeData:
    """ Holds the average or total time for each pipeline."""
    bio3: timedelta = timedelta()
    bio4: timedelta = timedelta()
    jams: timedelta = timedelta()
    wgsa2: timedelta = timedelta()
    woltka: timedelta = timedelta()
    sunbeam: timedelta = timedelta()

@dataclass
class ThreadData:
    """ Holds the number of CPUS used in each run. """
    bio3: int = 0
    bio4: int = 0
    jams: int = 0
    wgsa2: int = 0
    woltka: int = 0
    sunbeam: int = 0

In [3]:
def search_for_files(path: str):
    """ Look for anadama.log, JAMS logs, WGSA logs, and Wolka logs."""
    data_obj = OutputLogs()

    for root, dirs, files in os.walk(path):
        # print(files)
        for f in files:
            if f == "anadama.log":
                if "bio4" in root:
                    data_obj.bio4 = os.path.join(root, f)
                elif "bio3" in root:
                    data_obj.bio3 = os.path.join(root, f)
            elif f.endswith("JAMS.log"):
                # Don't add the beta log or the negative control log.
                if "beta" in root or "Neg" in f:
                    continue
                else:
                    data_obj.jams.append(os.path.join(root, f))
            elif f == "logfile.txt":
                data_obj.wgsa2 = os.path.join(root, f)

            elif f == "classify_time.log" or f == "bowtie_time.log":
                data_obj.woltka.append(os.path.join(root, f))
            
            elif re.match("\d+_benchmarks.tsv", f):
                data_obj.sunbeam = os.path.join(root, f)

    return data_obj


In [4]:
anadama_format = "%Y-%m-%d %H:%M:%S,%f"
wanted_time_fmt = "%H:%M:%S"


def parse_bio_time(log_path: str):
    """Parse the anadama.log file to get the start and end times."""
    # First line is the start time, last line is the end time.

    with open(log_path, 'r') as f:
        lines = f.readlines()
        start = datetime.strptime(
            lines[0].strip().split('\t')[0], anadama_format)
        end = datetime.strptime(
            lines[-1].strip().split('\t')[0], anadama_format)
        
        # Find the line that has "threads" in it.
        threads = 0
        for line in lines:
            if "threads" in line:
                # Get the number of threads used.
                threads = int(line.split(' ')[-1])
                break

        elapsed = end - start

        return elapsed, threads


In [5]:
jams_format_time = "%Y-%m-%d %H:%M:%S"


def average_time(times: list):
    """ Get the average time for each pipeline. """
    average = sum(times, timedelta()) / len(times)
    return average


def parse_jams_time(logs: list):
    """ Parse all of the JAMS logs. """

    cpu_regex = "Saving project workspace image using fastSave package with \d+ CPUs"

    times = []

    # They all ran with the same number of threads, so we can overwrite this in the for loop.
    threads = 0
    for l in logs:
        # open the file
        with open(l, 'r') as f:
            lines = f.readlines()
            # First line is start time.
            start = " ".join(lines[0].strip().split()[1:3]).strip("[]")
            start_time = datetime.strptime(start, jams_format_time)
            # Last line is end time.
            end = " ".join(lines[-1].strip().split()[1:3]).strip("[]")
            end_time = datetime.strptime(end, jams_format_time)

            for line in lines:
                if "Saving project workspace image using fastSave package with" in line:
                    threads = int(line.split()[-2])
                    break                  

            elapsed = end_time - start_time

            times.append(elapsed)

    return average_time(times), threads

In [6]:
def parse_wgsa2(file_path: str):
    """ Parse the WGSA output log for the time."""
    with open(file_path, 'r') as f:
        lines = f.readlines()
        start = " ".join(lines[0].strip().split()[0:2]).strip("[]")
        start_time = datetime.strptime(start, anadama_format)
        # Last line is end time.
        end = " ".join(lines[-1].strip().split()[0:2]).strip("[]")
        end_time = datetime.strptime(end, anadama_format)

        threads = 0
        for line in lines:
            if 'Provided cores:' in line:
                threads = int(line.split()[-1])
                break

        elapsed = end_time - start_time
        return elapsed, threads

In [7]:
def parse_woltka_time(logs: list):
    times = []
    for log in logs:
        # Second line gives column widths.
        with open(log, 'r') as f:
            dash_line = f.readlines()[1].strip().split()
            widths = [len(x)+1 for x in dash_line]

            df = pd.read_fwf(log, widths=widths, skiprows=[1], header=0)
            df = df.loc[df["JobName"] == "swarm"]

            # Split along the colon. This is of the format DD:HH:MM.
            df["Elapsed"] = df["Elapsed"].str.split(":").apply(
                lambda x: timedelta(days=int(x[0]), hours=int(x[1]), minutes=int(x[2])))

            avg_td = df["Elapsed"].mean()

            times.append(avg_td)

    # Sum of the bowtie and classify times.
    total_time = sum(times, timedelta())
    return total_time.to_pytimedelta()


In [8]:
def parse_sunbeam(file_path: str) -> Tuple[timedelta, int]:
    # Read in the file.
    df = pd.read_csv(file_path, sep="\t")

    display(df.head())

    # Get the total time.
    total_time = df["s"].sum()

    # All the sunbeam runs used 32 threads.
    return timedelta(seconds=total_time), 32

In [9]:
def parse_all_times(log_paths: OutputLogs) -> TimeData:
    """ Parses all time data, returns a TimeData object."""
    times = TimeData()
    threads_data = ThreadData()

    # Parse bio3 time.
    # bio3_time, threads = parse_bio_time(log_paths.bio3)
    # times.bio3 = bio3_time / threads
    # threads_data.bio3 = threads
    # Parse bio4 time.
    bio4_time, threads = parse_bio_time(log_paths.bio4)
    times.bio4 = bio4_time / threads
    # threads_data.bio4 = threads
    # Parse JAMS time.
    jams_times, threads = parse_jams_time(log_paths.jams)
    times.jams = jams_times / threads
    # threads_data.jams = threads
    # Parse WGSA2 time.
    # wgsa2_time, threads = parse_wgsa2(log_paths.wgsa2)
    # times.wgsa2 = wgsa2_time / threads
    # # threads_data.wgsa2 = threads
    # # Parse Woltka time.
    # woltka_times = parse_woltka_time(log_paths.woltka)
    # # Woltka used 16 threads. This could be automated but since we are doing just this one pipeline, it's fine.
    # times.woltka = woltka_times / 16

    sunbeam_times = parse_sunbeam(log_paths.sunbeam)
    times.sunbeam = sunbeam_times[0] / sunbeam_times[1]

    return times, threads_data


def analyze_times(log_paths: OutputLogs):
    times, threads = parse_all_times(log_paths)

    # print(asdict(times))
    # print(threads)

    time_df = pd.DataFrame(asdict(times), index=[0])
    # Drop the second row.
    time_df.drop(["bio3", "wgsa2", "woltka"], axis=1, inplace=True)

    display(time_df)

    # Relative difference is (x2 - min) / min
    relative_times = time_df.apply(lambda x: (
        x - time_df.min(axis=1)) / time_df.min(axis=1) * 100)

    # Set index value to be: "Relative Time (Factor of Smallest Time)"
    relative_times.index = ["Relative Time Per CPU (%)"]

    return time_df, relative_times


In [10]:
paths_dict = {
    # "nist": "/Volumes/TBHD_share/valencia/pipelines/NIST/"
    "tourlousse": "/Volumes/TBHD_share/valencia/pipelines/microbio_spectrum/CLEANED/pipelines/",
    "hilo": "/Volumes/TBHD_share/valencia/pipelines/amos/nibsc/hilo/",
    "mixed": "/Volumes/TBHD_share/valencia/pipelines/amos/nibsc/mixed/",
    "mbarc": "/Volumes/TBHD_share/valencia/pipelines/MBARC/pipelines/"
}

def main():
    final_df = pd.DataFrame()
    for k in paths_dict.keys():
        log_paths = search_for_files(paths_dict[k])
        # print(log_paths)

        time_df, relative_time_df = analyze_times(log_paths)

        display(time_df)

        # Format timedf to HH:MM:SS
        time_df = time_df.applymap(lambda x: str(x).split(".")[0])

        # Strip the days from the raw time_df.
        time_df = time_df.applymap(lambda x: x.split(" ")[-1])
        time_df.index = [k]

        # Concat the two dataframes.
        final_df = pd.concat([time_df, final_df], axis=0)

        display(time_df)

    final_df.to_latex("time_table.tex", index=True, escape=True)

main()


,rule,s,h:m:s,max_rss,max_vms,max_uss,max_pss,io_in,io_out,mean_load,cpu_time
0,adapter_removal_paired_SRR17380241,2234.4123,0:37:14,232.74,1494.91,162.72,170.61,0.0,0.0,107.90,2122.75
1,adapter_removal_paired_SRR17380242,2326.6287,0:38:46,226.06,1494.29,154.12,162.22,0.0,0.0,107.20,2207.32
2,adapter_removal_paired_SRR17380243,2284.9898,0:38:04,231.95,1494.62,167.41,174.25,0.0,0.0,108.04,2180.45
3,adapter_removal_paired_SRR17380244,2635.3709,0:43:55,225.63,1487.82,155.43,163.32,0.0,0.0,106.99,2531.41
4,adapter_removal_paired_SRR17380245,2392.7705,0:39:52,234.65,1497.52,164.26,172.13,0.0,0.0,108.15,2298.06


,bio4,jams,sunbeam
0,0 days 00:02:26.316828,0 days 00:02:35.404762,0 days 00:28:19.550388


,bio4,jams,sunbeam
0,0 days 00:02:26.316828,0 days 00:02:35.404762,0 days 00:28:19.550388


,bio4,jams,sunbeam
tourlousse,00:02:26,00:02:35,00:28:19


,rule,s,h:m:s,max_rss,max_vms,max_uss,max_pss,io_in,io_out,mean_load,cpu_time
0,adapter_removal_paired_SRR11487931,714.9280,0:11:54,232.62,1496.20,161.37,169.42,0.0,0.00,102.74,685.36
1,adapter_removal_paired_SRR11487932,782.5954,0:13:02,228.71,1495.50,157.09,165.20,0.0,0.00,101.65,744.38
2,adapter_removal_paired_SRR11487933,653.3541,0:10:53,234.56,1496.94,163.12,171.20,0.0,0.01,103.08,625.00
3,adapter_removal_paired_SRR11487934,664.4705,0:11:04,233.49,1497.37,162.21,170.31,0.0,0.00,101.88,627.11
4,adapter_removal_paired_SRR11487935,582.7689,0:09:42,229.18,1492.29,157.74,165.79,0.0,0.00,101.12,537.79


...

,rule,s,h:m:s,max_rss,max_vms,max_uss,max_pss,io_in,io_out,mean_load,cpu_time
0,adapter_removal_paired_MBARC,49033.1799,13:37:13,236.17,1497.99,174.27,183.67,0.0,0.01,110.76,47251.28
1,align_to_host_GRCh38.p14_genomic_MBARC,21593.9965,5:59:53,5919.91,6345.84,5917.34,5917.49,0.0,0.00,404.83,87417.64
2,classic_k2_biom,6.8183,0:00:06,95.75,525.22,92.84,93.04,0.0,0.00,17.76,1.07
3,filter_unmapped_reads_MBARC_1,29039.5720,8:03:59,186938.52,189909.02,186931.00,186932.50,0.0,0.00,99.52,28897.44
4,filter_unmapped_reads_MBARC_2,26053.6807,7:14:13,184008.84,187027.02,184001.19,184002.69,0.0,0.00,99.53,25930.58


,bio4,jams,sunbeam
0,0 days 00:23:10.696281,0 days 00:44:14.812500,0 days 01:44:40.140616


,bio4,jams,sunbeam
0,0 days 00:23:10.696281,0 days 00:44:14.812500,0 days 01:44:40.140616


,bio4,jams,sunbeam
mbarc,00:23:10,00:44:14,01:44:40


/var/folders/22/_2115q192kx7c1z764lwhkth898844/T/ipykernel_37180/2409326690.py:31: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  final_df.to_latex("time_table.tex", index=True, escape=True)


In [11]:
def test_nist_data():
    # Assert that the NIST output is TimeData(bio3=datetime.timedelta(seconds=28415, microseconds=815000), bio4=datetime.timedelta(seconds=4478, microseconds=860000), jams=datetime.timedelta(seconds=5213, microseconds=800000), wgsa2=datetime.timedelta(seconds=4963, microseconds=465000), woltka=datetime.timedelta(seconds=25692))).

    wanted = TimeData(bio3=timedelta(seconds=28415, microseconds=815000), bio4=timedelta(seconds=4478, microseconds=860000), jams=timedelta(
        seconds=5213, microseconds=800000), wgsa2=timedelta(seconds=4963, microseconds=465000), woltka=timedelta(seconds=25692))
    log_path = search_for_files(paths_dict["nist"])
    times, times_df = analyze_times(log_path)

    assert times == wanted


test_nist_data()


KeyError: 'nist'